# Source to text

In [ ]:
import pandas as pd

import os, sys
import numpy as np
import re
import six
import glob

from openpyxl import Workbook

In [ ]:
# from google BERT
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")

# Split sentences

In [ ]:
def getKoreanOnly(text):
    hangulonly = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangulonly.sub('', text)
    return result

def getEnglishOnly(text):
    enonly = re.compile('[^ a-zA-Z]+')
    result = enonly.sub('', text)
    return result

def parseRule1(lines):
    lines = lines.split('\n')
    
    regexp_rear = re.compile('(니다\.?|었음\.?|였음\.?|있음\.?(?!\)|없음\.?|'\
                        '행함\.?|렸다(\.|,)?)|옴\.|받음\.|호전(됨)?\.|임\.|짐\.|았음\.|퇴원함\.|'\
                        '다고 함\.|발생함\.|함\.|됨\.)')
    regexp_front = re.compile('\*검사')
    
    newlines = []
    linestack = ""
    for line in lines:

        korOnlyTmp = getKoreanOnly(line).strip()
        engOnlyTmp = getEnglishOnly(line).strip()
        if len(korOnlyTmp)<2 and len(engOnlyTmp)<2:
            newlines.append(line.strip())
            continue
        
        # lines of hangul        
        if len(korOnlyTmp)>2:
            if (('안녕' in line) and ('선생님' in line)):
                if linestack!='':
                    iter = re.finditer(regexp_rear, linestack)
                    indices_r = [m.end(0) for m in iter]
                    iter = re.finditer(regexp_front, linestack)
                    indices_f = [m.start(0) for m in iter]
                    indices = []
                    for i in range(len(indices_r)):
                        indices.append(indices_r[i])
                    for i in range(len(indices_f)):
                        indices.append(indices_f[i])
                    indices = np.array(indices)
                    indices.sort()
                    indices = np.unique(indices)
                    
                    lastidx = 0
                    for i in range(len(indices)):
                        linetmp = textsep(linestack[lastidx:indices[i]].strip())
                        if linetmp!="":
                            newlines.append(linetmp)
                        lastidx=indices[i]
    
                    if lastidx!=len(linestack):
                        linetmp = textsep(linestack[lastidx:].strip())
                        if linetmp!="":
                            newlines.append(linetmp)
                    linestack=""
    
                linetmp = textsep(line.strip())
                newlines.append(linetmp)
                continue
            
            else:  
                linestack = linestack.strip() + ' ' + line.strip()
                
        # non-Hangul lines
        else:
            if linestack!='':
                iter = re.finditer(regexp_rear, linestack)
                indices_r = [m.end(0) for m in iter]
                iter = re.finditer(regexp_front, linestack)
                indices_f = [m.start(0) for m in iter]
                indices = []
                for i in range(len(indices_r)):
                    indices.append(indices_r[i])
                for i in range(len(indices_f)):
                    indices.append(indices_f[i])
                indices = np.array(indices)
                indices.sort()
                indices = np.unique(indices)
                
                lastidx = 0
                for i in range(len(indices)):
                    linetmp = textsep(linestack[lastidx:indices[i]].strip())        
                    if linetmp!="":
                        newlines.append(linetmp)
                    lastidx=indices[i]
                
                if lastidx!=len(linestack):
                    linetmp = textsep(linestack[lastidx:].strip())
                    if linetmp!="":
                        newlines.append(linetmp)
                linestack=""

            # add current line
            linetmp = textsep(line.strip())
            newlines.append(linetmp)
    
    
    if linestack!='':
        # linestack 에 쌓인 문장들을 구분
        iter = re.finditer(regexp_rear, linestack)
        indices_r = [m.end(0) for m in iter]
        iter = re.finditer(regexp_front, linestack)
        indices_f = [m.start(0) for m in iter]
        indices = []
        for i in range(len(indices_r)):
            indices.append(indices_r[i])
        for i in range(len(indices_f)):
            indices.append(indices_f[i])
        indices = np.array(indices)
        indices.sort()
        indices = np.unique(indices)
                
        lastidx = 0
        for i in range(len(indices)):
            linetmp = textsep(linestack[lastidx:indices[i]].strip())
            if linetmp!="":
                newlines.append(linetmp)
            lastidx=indices[i]
    
        if lastidx!=len(linestack):
            linetmp = textsep(linestack[lastidx:].strip())
            if linetmp!="":
                newlines.append(linetmp)
        linestack=""
        
    return '\n'.join(newlines)


# add whitespaces

In [ ]:
def whitespace_en_ko(text):
    sep_en_ko = re.compile('[a-zA-Z][가-힇]') 
    iter = re.finditer(sep_en_ko, text)
    indices = [m.end(0) for m in iter]
    
    lastidx = 0
    seperatedtext=[]
    if len(indices) >= 1:
        lastidx = 0
        for i in range(len(indices)):
            seperatedtext.append(text[lastidx:indices[i]-1].strip())
            lastidx=indices[i]-1
        seperatedtext.append(text[lastidx:len(text)].strip())
        
        text = " ".join(seperatedtext)

    # 한글 영문
    sep_ko_en = re.compile('[가-힇][a-zA-Z]') 
    iter = re.finditer(sep_ko_en, text)
    indices = [m.end(0) for m in iter]

    lastidx = 0
    seperatedtext=[]
    if len(indices) >= 1:
        lastidx = 0
        for i in range(len(indices)):
            seperatedtext.append(text[lastidx:indices[i]-1].strip())
            lastidx=indices[i]-1
        seperatedtext.append(text[lastidx:len(text)].strip())
        
        text = " ".join(seperatedtext)
        
    return text

whitespace_en_ko("그러나ER에서 곧바로 ICU 전동함   올라온 직후, cardiac arrest    -> CPR, but not resuscitation   -> expire")
whitespace_en_ko(text)

In [ ]:
import re

def whitespace_number_spchar_char(text):
    sep_en_ko = re.compile('[a-zA-Z가-힇][.0-9 \(\{\[\-_\]\}\]\\\/]') 
    iter = re.finditer(sep_en_ko, text)
    indices = [m.end(0) for m in iter]
    
    lastidx = 0
    seperatedtext=[]
    if len(indices) >= 1:
        lastidx = 0
        for i in range(len(indices)):
#             print(text[lastidx:indices[i]-1])
            seperatedtext.append(text[lastidx:indices[i]-1].strip())
            lastidx=indices[i]-1
        seperatedtext.append(text[lastidx:len(text)].strip())
        
        text = " ".join(seperatedtext)
#         print("text: ", text)

    
    sep_ko_en = re.compile('[.0-9 \(\{\[\-_\]\}\]\\\/][a-zA-Z가-힇]') 
    iter = re.finditer(sep_ko_en, text)
    indices = [m.end(0) for m in iter]

    lastidx = 0
    seperatedtext=[]
    if len(indices) >= 1:
        lastidx = 0
        for i in range(len(indices)):
#             print(text[lastidx:indices[i]-1])
            seperatedtext.append(text[lastidx:indices[i]-1].strip())
            lastidx=indices[i]-1
        seperatedtext.append(text[lastidx:len(text)].strip())
        
        text = " ".join(seperatedtext)
#         print("text: ", text)
        
    return text

text = "Ultracet ER tab(Acetaminopehn 650mg/Tramadol 75mg) 1tab [P.O] daily pc 많이 아플 때 추가로 드세요 X10 Days"
whitespace_number_spchar_char(text)

In [ ]:
def whitespace_num_day(text):
    days_sep = re.compile('[ 0-9]+일\s+[ 0-9]+회\s+\*\s+[ 0-9]+일') 
    
    iter = re.finditer(days_sep, text)
    indices = [m.end(0) for m in iter]
    
    seperatedtext=[]
    if len(indices) >= 1:
        lastidx = 0
        for i in range(len(indices)):
            seperatedtext.append(text[lastidx:indices[i]].strip())
            lastidx=indices[i]
    else:
        return text
    
    return '\n'.join(seperatedtext)

### special characters

In [ ]:
import re
def textsep(text):
    text = re.sub(r'(?<=\()\+', ' + ', text)
    text = re.sub(r'\+(?=\))', ' + ', text)
    text = re.sub(r'(?<=/)\+(?=/)', ' + ', text)
    text = re.sub(r'\(\+\)', ' ( + ) ', text)
    text = re.sub(r'(?<=/)\+', ' + ', text)  # /+ => / +
    text = re.sub(r'\+(?=/)', ' + ', text)
    text = re.sub(r'\s+\+\s+', ' + ', text) #+ 를 ' + ' 으로
    
    text = re.sub(r'(?<=\()\-', ' - ', text)
    text = re.sub(r'\-(?=\))', ' - ', text)
    text = re.sub(r'(?<=/)\-(?=/)', ' - ', text)
    text = re.sub(r'\(\-\)', ' ( - ) ', text)
    text = re.sub(r'(?<=/)\-', ' - ', text)  # /- => / -
    text = re.sub(r'\-(?=/)', ' - ', text)
    text = re.sub(r'\s+\-\s+', ' - ', text) #-를 ' - ' 으로
    
    text = re.sub(r'\s?\(\s?', ' ( ', text)
    text = re.sub(r'\s?\)\s?', ' ) ', text)
    
    # date expression with - 
    text = re.sub(r'(?<=\d)\-(?=\d)', ' - ', text)
    text = re.sub(r'(?<=\d)\/(?=\d)', ' / ', text)
    text = re.sub(r'(?<=\d)\.(?=\d)', ' . ', text)
        
    text = re.sub(r'x ray', 'x-ray ', text)
    text = re.sub(r'X ray', 'X-ray ', text)
    
    # P/E>   P/E : 
    text = re.sub(r'(?<=[.,a-zA-Z])\>', ' > ', text)
    
    # ,    ' , '
    text = re.sub(r'\,', ' , ', text)
    text = re.sub(r'\s+,\s+', ' , ', text) #, 를 ' , ' 으로
    
    # :  ' : '
    text = re.sub(r'(?<=[a-zA-Z])\:', ' : ', text)
    text = re.sub(r'\s+\:\s+', ' : ', text) #, 를 ' , ' 으로
    
    text = re.sub(r'#\s?[0-9]+\.\s+', '', text)
    
    text = re.sub(r'\-{5,}', '', text)
    text = re.sub(r'\={5,}', '', text)
    
    text = re.sub(r'\(', ' ( ', text)
    text = re.sub(r'\s+\(\s+', ' ( ', text)
    
    text = re.sub(r'\)', ' ) ', text)
    text = re.sub(r'\s+\)\s+', ' ) ', text)
    
    text = re.sub(r'\:', ' : ', text)
    text = re.sub(r'\s+\:\s+', ' : ', text)
    
    text = re.sub(r';', ' ; ', text)
    text = re.sub(r'\s+;\s+', ' ; ', text)
    
    text = re.sub(r'(?<=[0-9])일', ' 일', text)
    text = re.sub(r'(?<=[0-9])회', ' 회', text)

    text = re.sub(r'tab', ' tab ', text)
    
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', '\t', text)
    
    return text


text = "수진 ( 진료 ) 일 : 2011-01-13 진료과 : 소화기내과 Description <- 소견 및 계획 : s )"
text_ = textsep(text)
print(text)
print(text_)

## deidentification
- After delete information automatically based on rulebase, manually deleted sensitive contents. 

In [ ]:
def deidentificaiton(text):    
    text = re.sub(r'([가-힇])+(\s+)?배상', 'OOO 배상', text)
    text = re.sub(r'([가-힇])+(\s+)?교수', 'OOO 교수', text)
    text = re.sub(r'([가-힇])+(\s+)?선생님', 'OOO 선생님', text)
    text = re.sub(r'pf(\s+)?([가-힇])+', 'pf OOO', text)
    text = re.sub(r'pf\.(\s+)?([가-힇])+', 'pf. OOO', text)
    text = re.sub(r'Pf\.(\s+)?([가-힇])+', 'pf. OOO', text)
    text = re.sub(r'PA(\s+)?([가-힇])+', 'PA OOO', text)

    return text

text = "PA 김경모"
deidentificaiton(text)

In [ ]:
def delete_enter(lines):
    del_entered = []
    lines = str(lines).split("\n")
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        line = lines[l].strip()
        if line=="":
            continue
        else:
            del_entered.append(line)
    
    newlines = "\n".join(del_entered)
    
    return newlines

lines = "GW으로 검사하심\n\n\n낮에 누가 집에 다녀간다"
delete_enter(lines)

In [ ]:
# all tab changed to "    "
def tab_to_spaces(line):
    line = line.replace("\t", "    ")
    return line
tab_to_spaces("Uric acid	5 . 0")

In [ ]:
def text_preprocessing(lines):
    if len(str(lines).strip("\n").strip())==0:
        return []
    data = []
    lines = str(lines).split("\n")
    for l in range(len(lines)):
        line = lines[l].strip()
        line = tab_to_spaces(line)
        line = delete_enter(line)
        line = parseRule1(line)
        line = textsep(line)

        line = deidentificaiton(line)
        line = whitespace_en_ko(line)
        line = whitespace_number_spchar_char(line)        
        line = delete_enter(line)
        line = line.strip()
        if line=="":
            continue
        data.append(line)
        
    return data

# Merge section

In [ ]:
import time
def runloop(ws, outputpath):
    start_time = time.time()

    doccount = 0
    docperfile = 50

    docout_lastnum = 0
    documents = ""
    
    for row in range(0, ws.shape[0]):
        pt_current = ws['fake_id'][row] # ubuntu
        dt_current = ws['수진(진료)일'][row]
        dp_current = ws['서식명'][row]
        se_current = ws['서식항목명'][row]
        
        dp_current = ws['서식명'][row]
        prep = text_preprocessing(dp_current)
        dp_current = " ".join(prep) # document species name should be in a row
        
        se_current = ws['서식항목명'][row]
        prep = text_preprocessing(se_current)
        se_current = " ".join(prep) # section name should be in a row
        
        data = ws['서식내용'][row] 
        prep = text_preprocessing(data)
        data = "\n".join(prep) # original text allow enter values
                
        # check the the target document
        # because target document lines are shown in multiple rows
        if row==0:
            documents = "서식명 : "+dp_current+" 진료일 : "+textsep(dt_current) + "\n"
            documents = documents+se_current + " : "+data + "\n"
        
        elif pt_current==pt_last and dt_last==dt_current and dp_current==dp_last:
            documents = documents + se_current + " : "+data + "\n"
        
        else:
            doccount = doccount+1

            target_folder = output_directory+"/"+str(((pt_last//docperfile)+1)*docperfile)
            if not os.path.exists(target_folder):
                os.makedirs(target_folder)
            target_txt = str("".join(['0']*(8-len(str(pt_last)))) + str(int(pt_last)))
            dout = target_folder+"/"+str(target_txt)+".txt"

            f = open(dout, "a")
            f.write(documents+"\n")
            f.close()

            documents = "서식명 : "+dp_current+" 진료일 : "+textsep(dt_current) + "\n"
            documents = documents+se_current + " : "+data + "\n"
        
        # prepare next step
        pt_last = pt_current
        dp_last = dp_current
        dt_last = dt_current
        
    # rest
    if documents!='':
        target_folder = output_directory+"/"+str(((pt_last//docperfile)+1)*docperfile)
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)
        target_txt = str("".join(['0']*(8-len(str(pt_last)))) + str(int(pt_last)))
        dout = target_folder+"/"+str(target_txt)+".txt"
        
        f = open(dout, "a")
        f.write(documents+"\n\n")
        f.close()
        
        documents=""

    #write time lap
    file1 = open("sentenceSplitTimelap.txt","a")
    file1.write("\n--- %s finished seconds ---" % (time.time() - start_time))
    file1.close()

In [ ]:
output_directory = "./SNUH_visit_2011to2020"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

filenames = glob.glob('../sources/visits_2011to2020/*.xlsx')
filenames.sort()
print(filenames)
for f in range(len(filenames)):
    reading_file = filenames[f].split("/")[-1]
    print("reading_file: ", reading_file)
    
    print("reading...")
    ws = pd.read_excel(filenames[f])
    
    print("processing...")
    runloop(ws, outputpath=output_directory)

print("Done")